In [1]:
from fastembed import TextEmbedding

In [2]:
query = 'I just discovered the course. Can I join now?'

In [3]:
from qdrant_client import QdrantClient, models

In [4]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [5]:
import json

EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [8]:
embedder = TextEmbedding(model_name=model_handle)

In [19]:
q, = list(embedder.embed(query))

In [11]:
q.min()

np.float64(-0.11726373885183883)

In [12]:
import numpy as np
np.linalg.norm(q)

np.float64(1.0)

In [22]:
q.dot(q)

np.float64(1.0000000000000002)

In [14]:
doc = 'Can I still join the course after the start date?'

In [30]:
q2, = list(embedder.embed(doc))

In [31]:
q.dot(q2)

np.float64(0.9008528895674548)

In [32]:
#Q3. Ranking by cosine

In [58]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [59]:

#solution 1
import pandas as pd

df = pd.DataFrame(documents)
V_text = list(embedder.embed(df.text))
V_text = np.array(V_text)

In [63]:
# Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.
#solution 2
V = []

for doc in documents:
    vector_input =  doc['question'] + ' ' + doc['text']
    q, = list(embedder.embed(vector_input))
    V.append(q)

In [64]:
np.array(V).dot(q)

array([0.76691052, 0.81023566, 0.80029351, 0.79552661, 1.        ])

In [62]:
V_text = list(embedder.embed(df.question + ' ' + df.text))
V_text = np.array(V_text)

similarity = V_text.dot(q)
similarity.argmax()

np.int64(4)

In [ ]:
# Q5. Selecting the embedding model